<a href="https://colab.research.google.com/github/yugamjayant/Sentiment-analysis-twitter/blob/master/Movie_recommendation_sys.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import numpy.ma as ma
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
import tabulate

In [4]:
links_rawds = pd.read_csv("/content/drive/MyDrive/Colab_Notebooks/ml-latest-small/links.csv")
movies_rawds = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/ml-latest-small/movies.csv')
ratings_rawds = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/ml-latest-small/ratings.csv')
tags_rawds = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/ml-latest-small/tags.csv')

Data Preprocessing

In [110]:
#Lets prepare data that can be fed to the model
#Lets start by preaparing User side of data

movie_genres = movies_rawds

all_genres = set()
for index, row in movies_rawds.iterrows():
  all_genres.update(row['genres'].split('|'))

#Remove "(no genres listed)" from genres list
all_genres.discard('(no genres listed)')


#Adding new columns, denoting presence of each genre for each movie

for genre in all_genres:
  movie_genres[genre] = movie_genres['genres'].str.contains(genre)


#Merging the dataframes, rating rewards dataframe and movie genres dataframe

ratings_movie_genres = pd.merge(left = ratings_rawds[['userId', 'movieId', 'rating']],
                                right = movie_genres,
                                on = 'movieId',
                                how = 'left')


#Function that renames columns, it helps in assigning new column names after aggregatte function
def create_col_names(agg_dict, agg_col):
  col_names = [agg_col]
  for key in agg_dict.keys():
    value = agg_dict[key]
    if type(value) is list:
      for value_1 in value:
        col_names.append(key+'_'+value_1)
    elif type(value) != str:
      col_names.append(key+'_'+'cfunc')
    else:
      col_names.append(key+'_'+value)

  return col_names


#This function calculates genre wise avr ratings
def genre_wise_avg_rating(array_temp):

  sum_ratings = array_temp.sum()
  #Non zero ratings count
  array_temp_summary = array_temp.value_counts()
  non_zero_ratings_count = array_temp_summary.sum()

  if 0 in array_temp_summary.index:
    non_zero_ratings_count = array_temp_summary.sum() - array_temp_summary[0]

  return sum_ratings/non_zero_ratings_count



#Adding new columns, denoting presence of each genre for each movie

for genre in all_genres:
  movie_genres[genre] = movie_genres['genres'].str.contains(genre)



#Merging the dataframes, rating rewards dataframe and movie genres dataframe

ratings_movie_genres = pd.merge(left = ratings_rawds[['userId', 'movieId', 'rating']],
                                right = movie_genres,
                                on = 'movieId',
                                how = 'left')


#Function that renames columns, it helps in assigning new column names after aggregatte function
def create_col_names(agg_dict, agg_col):
  col_names = [agg_col]
  for key in agg_dict.keys():
    value = agg_dict[key]
    if type(value) is list:
      for value_1 in value:
        col_names.append(key+'_'+value_1)
    elif type(value) != str:
      col_names.append(key+'_'+'cfunc')
    else:
      col_names.append(key+'_'+value)

  return col_names


global user_genre_ratings_array

#This function calculates genre wise avr ratings
def genre_wise_avg_rating(array_temp):
  global user_genre_ratings_array
  user_genre_ratings_array = array_temp
  sum_ratings = array_temp.sum()
  #Non zero ratings count
  array_temp_summary = array_temp.value_counts()
  non_zero_ratings_count = array_temp_summary.sum()

  if 0 in array_temp_summary.index:
    non_zero_ratings_count = array_temp_summary.sum() - array_temp_summary[0]

  return sum_ratings/non_zero_ratings_count

#Adding rating of each movie to genre column, to facilitate averages in thenext step
for genre in all_genres:
  ratings_movie_genres[genre] = ratings_movie_genres[genre].astype(float) * ratings_movie_genres['rating']


#Aggregating the dataframe to compute average ratings for each genre
movie_genre_df_agg_dict = {'rating':['mean', 'sum'], 'movieId':'count', 'War': genre_wise_avg_rating, 'Adventure': genre_wise_avg_rating, 'Romance': genre_wise_avg_rating, 'Musical': genre_wise_avg_rating, 'Western': genre_wise_avg_rating, 'Fantasy': genre_wise_avg_rating, 'Children': genre_wise_avg_rating, 'Sci-Fi': genre_wise_avg_rating, 'Drama': genre_wise_avg_rating, 'Crime': genre_wise_avg_rating, 'Mystery': genre_wise_avg_rating, 'Horror': genre_wise_avg_rating, 'Documentary': genre_wise_avg_rating, 'Action': genre_wise_avg_rating, 'Comedy': genre_wise_avg_rating, 'Thriller': genre_wise_avg_rating, 'Animation': genre_wise_avg_rating, 'Film-Noir': genre_wise_avg_rating, 'IMAX': genre_wise_avg_rating}

ratings_movie_genres_int = ratings_movie_genres.groupby('userId').aggregate(movie_genre_df_agg_dict).reset_index()

ratings_movie_genres_int.columns = create_col_names(movie_genre_df_agg_dict, 'userId')

ratings_movie_genres_int.fillna(0, inplace = True)


#Expanding the df such that we our number of rows matches the number of ratings
ratings_movie_genres_int2 = pd.merge(ratings_rawds[['userId']], ratings_movie_genres_int, on = 'userId', how = 'left')


#Excluded few unnecessary columns
user_ratings_df = ratings_movie_genres_int2[['userId','movieId_count', 'rating_mean', 'Action_cfunc', 'Western_cfunc', 'Sci-Fi_cfunc', 'IMAX_cfunc', 'Drama_cfunc',
       'Thriller_cfunc', 'Adventure_cfunc', 'Animation_cfunc', 'Musical_cfunc', 'Mystery_cfunc', 'Fantasy_cfunc',
       'Children_cfunc', 'Film-Noir_cfunc', 'Romance_cfunc', 'Crime_cfunc', 'War_cfunc', 'Documentary_cfunc',
       'Horror_cfunc', 'Comedy_cfunc']]


#Now Preparing item side of dataframe







#We needed year in the dataframe, thus the following function extracts year from movie title

def extract_year(title):
  try:
    year = re.search("\(([0-9]{4})\)", title).group().strip('()')

  except Exception as e:
    print("Error: ", e)
    year = 0

  return int(year)


movie_genres['year'] = movie_genres['title'].apply(lambda x: extract_year(x))


#Expanding the df such that we our number of rows matches the number of ratings

movie_genres_ratings = pd.merge(ratings_rawds[['movieId']], movie_genres, on = 'movieId', how = 'left')

for genre in all_genres:
  movie_genres_ratings[genre] = movie_genres_ratings[genre].astype(int)


movie_genres_ratings.drop(columns = ['genres'], inplace = True)

#calculating avg ratings for each movie
avg_ratings_movie_wise = ratings_rawds.groupby('movieId').aggregate({'rating':'mean'}).reset_index().rename(columns = {'rating':'mean_ratings'})

#Adding avg rating(per movie) column to movie_genres_ratings
movie_genres_ratings = pd.merge(movie_genres_ratings, avg_ratings_movie_wise[['movieId','mean_ratings']], on = 'movieId', how = 'left')

item_ratings_df = movie_genres_ratings[['movieId', 'year','mean_ratings','Action', 'Western', 'Sci-Fi', 'IMAX', 'Drama',
       'Thriller', 'Adventure', 'Animation', 'Musical', 'Mystery', 'Fantasy',
       'Children', 'Film-Noir', 'Romance', 'Crime', 'War', 'Documentary',
       'Horror', 'Comedy']]



<ipython-input-110-2bffd7de85c3>:104: RuntimeWarning: invalid value encountered in double_scalars
  return sum_ratings/non_zero_ratings_count
<ipython-input-110-2bffd7de85c3>:104: RuntimeWarning: invalid value encountered in double_scalars
  return sum_ratings/non_zero_ratings_count
<ipython-input-110-2bffd7de85c3>:104: RuntimeWarning: invalid value encountered in double_scalars
  return sum_ratings/non_zero_ratings_count
<ipython-input-110-2bffd7de85c3>:104: RuntimeWarning: invalid value encountered in double_scalars
  return sum_ratings/non_zero_ratings_count
<ipython-input-110-2bffd7de85c3>:104: RuntimeWarning: invalid value encountered in double_scalars
  return sum_ratings/non_zero_ratings_count
<ipython-input-110-2bffd7de85c3>:104: RuntimeWarning: invalid value encountered in double_scalars
  return sum_ratings/non_zero_ratings_count
<ipython-input-110-2bffd7de85c3>:104: RuntimeWarning: invalid value encountered in double_scalars
  return sum_ratings/non_zero_ratings_count
<ipyth

Error:  'NoneType' object has no attribute 'group'
Error:  'NoneType' object has no attribute 'group'
Error:  'NoneType' object has no attribute 'group'
Error:  'NoneType' object has no attribute 'group'
Error:  'NoneType' object has no attribute 'group'
Error:  'NoneType' object has no attribute 'group'
Error:  'NoneType' object has no attribute 'group'
Error:  'NoneType' object has no attribute 'group'
Error:  'NoneType' object has no attribute 'group'
Error:  'NoneType' object has no attribute 'group'
Error:  'NoneType' object has no attribute 'group'
Error:  'NoneType' object has no attribute 'group'
Error:  'NoneType' object has no attribute 'group'


In [124]:
#Creating user and item numpy arrays, here we have sliced the dataframe to remove the index column

item_train = item_ratings_df.to_numpy()[:,1:]
user_train = user_ratings_df.to_numpy()[:,1:]